In [ ]:
"""
ULTRA-ENHANCED PPM + ADVANCED TECHNIQUES
Target: 0.650-0.670 AUC

Improvements:
1. Multi-headed PPM (different attention patterns)
2. Feature interactions via FiBiNet
3. Enhanced statistical features
4. Adversarial validation-aware training
5. Prediction calibration
"""

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.nn.init as init
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
import gc
import math

print("=" * 80)
print("ULTRA-ENHANCED PPM MODEL")
print("Multi-head PPM + FiBiNet + Advanced Features")
print("Target: 0.650-0.670 AUC")
print("=" * 80)

CONFIG = {
    'data_folder': '/kaggle/input',
    'output_folder': '/kaggle/working/',
    'train_path': '/interactions-data/train_interactions.parquet',
    'test_path': '/new-test-dataset/test_pairs_public.parquet',
    'items_meta_path': '/interactions-data/items_meta.parquet',
    'users_meta_path': '/interactions-data/users_meta.parquet',
    'test_output_path': 'ultra_enhanced_ppm',
    
    # Enhanced architecture
    'emb_size': 192,
    'stat_emb_size': 64,  
    'deep_layers': [896, 448, 224], 
    'num_cross_layers': 4,
    'dropout': 0.18,
    
    # PPM 
    'ppm_hidden_dim': 320,
    'ppm_num_layers': 2,
    'ppm_num_heads': 4,  # Multi-head attention
    'sequence_length': 20,  # Longer history
    
    # FiBiNet config
    'use_senet': True,
    'reduction_ratio': 3,
    
    # Training
    'DEVICE': 'cuda',
    'SEED': 42,
    'BATCH_SIZE': 14336, 
    'LR': 0.0022,
    'LR_MIN': 0.00003,
    'weight_decay': 6e-6,
    'EPOCHS': 1,
    'VAL_SPLIT': 0.05,
    'GRAD_CLIP': 1.0,
    'label_smoothing': 0.01,  # Regularization
}

device = torch.device(CONFIG['DEVICE'] if torch.cuda.is_available() else "cpu")
torch.manual_seed(CONFIG['SEED'])
np.random.seed(CONFIG['SEED'])

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()

print(f"\nDevice: {device}")

# DATA LOADING
print("\n[1/9] Loading data...")
train = pd.read_parquet(f"{CONFIG['data_folder']}{CONFIG['train_path']}", engine='pyarrow')
train['target'] = ((train['int1'] + train['int2'] + train['int3'] + train['int4']) > 0).astype('int8')

test = pd.read_parquet(f"{CONFIG['data_folder']}{CONFIG['test_path']}")
test_to_save = test.copy()

items_meta = pd.read_parquet(f"{CONFIG['data_folder']}{CONFIG['items_meta_path']}", engine='pyarrow')
items_meta['dur'] = items_meta['dur'] - 5
items_meta.set_index('iid', inplace=True)

users_meta = pd.read_parquet(f"{CONFIG['data_folder']}{CONFIG['users_meta_path']}", engine='pyarrow')
users_meta['age'] = users_meta['age'] - 18
users_meta['sex'] = users_meta['sex'].replace({1:0, 2:1})
users_meta.set_index('uid', inplace=True)

num_users = max(train['uid'].max(), test['uid'].max()) + 1
num_items = max(train['iid'].max(), test['iid'].max()) + 1
num_sources = items_meta['sid'].max() + 1
num_ages = users_meta['age'].max() + 1
num_genders = 2
num_durations = items_meta['dur'].max() + 1

print(f"  Train: {len(train):,}, Test: {len(test):,}")

# SEQUENCE BUILDING

print("\n[2/9] Building user sequences...")
import time
start_time = time.time()

user_groups = train.groupby('uid', sort=False)
user_sequences = {}

batch_size = 10000
unique_users = train['uid'].unique()
num_batches = (len(unique_users) + batch_size - 1) // batch_size

for batch_idx in tqdm(range(num_batches), desc="Building sequences"):
    start_idx = batch_idx * batch_size
    end_idx = min(start_idx + batch_size, len(unique_users))
    batch_users = unique_users[start_idx:end_idx]
    
    for uid in batch_users:
        user_data = user_groups.get_group(uid)
        user_sequences[uid] = {
            'items': user_data['iid'].values,
            'targets': user_data['target'].values,
            'length': len(user_data)
        }

print(f"  ✓ Built {len(user_sequences):,} sequences in {time.time()-start_time:.1f}s")

# ENHACED FEATURE ENGINEERING
print("\n[3/9] Computing enhanced features...")

# Item features with more statistics
item_stats = train.groupby('iid').agg({
    'target': ['sum', 'count', 'mean', 'std']
}).reset_index()
item_stats.columns = ['iid', 'item_pos_count', 'item_total_count', 'item_ctr', 'item_ctr_std']
item_stats['item_popularity'] = np.log1p(item_stats['item_total_count'])
item_stats['item_conversion_rate'] = item_stats['item_pos_count'] / (item_stats['item_total_count'] + 10)
item_stats['item_ctr_std'] = item_stats['item_ctr_std'].fillna(0)

# Percentile-based normalization (more robust)
for col in ['item_ctr', 'item_popularity', 'item_ctr_std', 'item_conversion_rate']:
    p01, p99 = item_stats[col].quantile([0.01, 0.99])
    item_stats[col] = np.clip(item_stats[col], p01, p99)
    item_stats[col] = (item_stats[col] - p01) / (p99 - p01 + 1e-8)

item_stats = item_stats[['iid', 'item_ctr', 'item_popularity', 'item_ctr_std', 'item_conversion_rate']].set_index('iid')

# User features with more statistics
user_stats = train.groupby('uid').agg({
    'target': ['sum', 'count', 'mean', 'std']
}).reset_index()
user_stats.columns = ['uid', 'user_pos_count', 'user_total_count', 'user_ctr', 'user_ctr_std']
user_stats['user_activity'] = np.log1p(user_stats['user_total_count'])
user_stats['user_engagement_rate'] = user_stats['user_pos_count'] / (user_stats['user_total_count'] + 10)
user_stats['user_ctr_std'] = user_stats['user_ctr_std'].fillna(0)

# Percentile normalization
for col in ['user_ctr', 'user_activity', 'user_ctr_std', 'user_engagement_rate']:
    p01, p99 = user_stats[col].quantile([0.01, 0.99])
    user_stats[col] = np.clip(user_stats[col], p01, p99)
    user_stats[col] = (user_stats[col] - p01) / (p99 - p01 + 1e-8)

user_stats = user_stats[['uid', 'user_ctr', 'user_activity', 'user_ctr_std', 'user_engagement_rate']].set_index('uid')

# User-source affinity
train_with_source = train.merge(items_meta[['sid']], left_on='iid', right_index=True, how='left')
user_source_stats = train_with_source.groupby(['uid', 'sid']).agg({
    'target': ['sum', 'count', 'mean']
}).reset_index()
user_source_stats.columns = ['uid', 'sid', 'us_pos', 'us_total', 'us_ctr']
user_source_stats['us_affinity'] = user_source_stats['us_pos'] / (user_source_stats['us_total'] + 5)
user_source_avg = user_source_stats.groupby('uid')['us_affinity'].mean().reset_index()
user_source_avg.columns = ['uid', 'user_avg_source_affinity']
user_source_max = user_source_stats.groupby('uid')['us_affinity'].max().reset_index()
user_source_max.columns = ['uid', 'user_max_source_affinity']

user_source_features = user_source_avg.merge(user_source_max, on='uid').set_index('uid')
user_stats = user_stats.join(user_source_features, how='left')
user_stats['user_avg_source_affinity'] = user_stats['user_avg_source_affinity'].fillna(0.5)
user_stats['user_max_source_affinity'] = user_stats['user_max_source_affinity'].fillna(0.5)

print(f"  Item features: {item_stats.shape[1]}")
print(f"  User features: {user_stats.shape[1]}")

del train_with_source, user_source_stats, user_source_avg, user_source_max, user_source_features
gc.collect()

val_size = int(len(train) * CONFIG['VAL_SPLIT'])
train_data = train.iloc[:-val_size].reset_index(drop=True)
val_data = train.iloc[-val_size:].reset_index(drop=True)
print(f"  Train: {len(train_data):,} | Val: {len(val_data):,}")

# ADVANCED MODEL COMPONENTS

print("\n[4/9] Defining model...")

class SENet(nn.Module):
    """Squeeze-and-Excitation Network for feature importance."""
    def __init__(self, num_fields, reduction_ratio=3):
        super().__init__()
        reduced_size = max(1, num_fields // reduction_ratio)
        self.excitation = nn.Sequential(
            nn.Linear(num_fields, reduced_size, bias=False),
            nn.ReLU(),
            nn.Linear(reduced_size, num_fields, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        # x: [batch, num_fields, emb_dim]
        batch_size, num_fields, emb_dim = x.size()
        
        # Squeeze: global average pooling
        squeeze = x.mean(dim=2)  # [batch, num_fields]
        
        # Excitation: learn field importance
        attention = self.excitation(squeeze)  # [batch, num_fields]
        
        # Reweight
        output = x * attention.unsqueeze(2)  # [batch, num_fields, emb_dim]
        return output

class BilinearInteraction(nn.Module):
    """Bilinear interaction from FiBiNet."""
    def __init__(self, emb_size):
        super().__init__()
        self.W = nn.Parameter(torch.randn(emb_size, emb_size))
        nn.init.xavier_uniform_(self.W)
    
    def forward(self, x):
        # x: [batch, num_fields, emb_dim]
        batch_size, num_fields, emb_dim = x.size()
        
        interactions = []
        for i in range(num_fields):
            for j in range(i+1, num_fields):
                # Bilinear: vi^T W vj
                vi = x[:, i, :]  # [batch, emb_dim]
                vj = x[:, j, :]  # [batch, emb_dim]
                interaction = (vi @ self.W * vj).sum(dim=1, keepdim=True)  # [batch, 1]
                interactions.append(interaction)
        
        if interactions:
            return torch.cat(interactions, dim=1)  # [batch, num_interactions]
        else:
            return torch.zeros(batch_size, 1, device=x.device)

class MultiHeadPPM(nn.Module):
    """Multi-head PPM for diverse attention patterns."""
    def __init__(self, emb_size, hidden_dim, num_layers, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.hidden_dim = hidden_dim
        
        # Multi-head item encoders
        self.item_encoders = nn.ModuleList([
            nn.Sequential(
                nn.Linear(emb_size, hidden_dim // num_heads),
                nn.LayerNorm(hidden_dim // num_heads),
                nn.ReLU(),
                nn.Dropout(CONFIG['dropout'])
            )
            for _ in range(num_heads)
        ])
        
        # Behavior RNN per head
        self.behavior_rnns = nn.ModuleList([
            nn.GRU(
                input_size=hidden_dim // num_heads,
                hidden_size=hidden_dim // num_heads,
                num_layers=num_layers,
                batch_first=True,
                dropout=CONFIG['dropout'] if num_layers > 1 else 0
            )
            for _ in range(num_heads)
        ])
        
        # Attention per head
        self.attentions = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_dim // num_heads, hidden_dim // num_heads),
                nn.Tanh(),
                nn.Linear(hidden_dim // num_heads, 1)
            )
            for _ in range(num_heads)
        ])
        
        # Fusion
        self.fusion = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(CONFIG['dropout'])
        )
    
    def forward(self, current_item_emb, sequence_embs, sequence_mask):
        batch_size = current_item_emb.size(0)
        
        head_outputs = []
        
        for head_idx in range(self.num_heads):
            # Encode current item
            current_enc = self.item_encoders[head_idx](current_item_emb)
            
            # Encode sequence
            seq_enc = self.item_encoders[head_idx](sequence_embs)
            
            # RNN
            rnn_out, _ = self.behavior_rnns[head_idx](seq_enc)
            
            # Attention
            attn_scores = self.attentions[head_idx](rnn_out).squeeze(-1)
            attn_scores = attn_scores.masked_fill(sequence_mask == 0, -1e9)
            attn_weights = F.softmax(attn_scores, dim=1)
            
            # Weighted sum
            seq_repr = torch.bmm(attn_weights.unsqueeze(1), rnn_out).squeeze(1)
            
            head_outputs.append(torch.cat([current_enc, seq_repr], dim=1))
        
        # Concatenate all heads
        combined = torch.cat(head_outputs, dim=1)
        output = self.fusion(combined)
        
        return output

class UltraEnhancedPPM(nn.Module):
    def __init__(self):
        super().__init__()
        emb_size = CONFIG['emb_size']
        stat_emb_size = CONFIG['stat_emb_size']
        ppm_dim = CONFIG['ppm_hidden_dim']
        
        # Embeddings
        self.user_emb_wide = nn.Embedding(num_users, emb_size)
        self.item_emb_wide = nn.Embedding(num_items, emb_size)
        self.source_emb_wide = nn.Embedding(num_sources, emb_size)
        self.age_emb_wide = nn.Embedding(num_ages, emb_size)
        self.gender_emb_wide = nn.Embedding(num_genders, emb_size)
        self.duration_emb_wide = nn.Embedding(num_durations, emb_size)
        
        self.user_emb_deep = nn.Embedding(num_users, emb_size)
        self.item_emb_deep = nn.Embedding(num_items, emb_size)
        self.source_emb_deep = nn.Embedding(num_sources, emb_size)
        self.age_emb_deep = nn.Embedding(num_ages, emb_size)
        self.gender_emb_deep = nn.Embedding(num_genders, emb_size)
        self.duration_emb_deep = nn.Embedding(num_durations, emb_size)
        
        self.item_meta_proj = nn.Linear(32, emb_size)
        
        # Enhanced stat projections (4 item + 6 user features)
        self.item_stat_proj = nn.Sequential(
            nn.Linear(4, stat_emb_size),
            nn.LayerNorm(stat_emb_size),
            nn.ReLU(),
            nn.Dropout(CONFIG['dropout'])
        )
        
        self.user_stat_proj = nn.Sequential(
            nn.Linear(6, stat_emb_size),
            nn.LayerNorm(stat_emb_size),
            nn.ReLU(),
            nn.Dropout(CONFIG['dropout'])
        )
        
        # Multi-head PPM
        self.ppm = MultiHeadPPM(
            emb_size=emb_size,
            hidden_dim=ppm_dim,
            num_layers=CONFIG['ppm_num_layers'],
            num_heads=CONFIG['ppm_num_heads']
        )
        
        # SENet for feature importance
        if CONFIG['use_senet']:
            self.senet = SENet(num_fields=7, reduction_ratio=CONFIG['reduction_ratio'])
        
        # Bilinear interactions
        self.bilinear = BilinearInteraction(emb_size)
        
        # Calculate dimensions
        num_bilinear = 7 * 6 // 2  # C(7,2) = 21 pairwise interactions
        wide_base = emb_size * 7 + stat_emb_size * 2 + ppm_dim + num_bilinear
        dcn_input = emb_size * 7 + stat_emb_size * 2 + ppm_dim
        
        # DCNv2
        self.cross_weights = nn.ParameterList([
            nn.Parameter(torch.randn(dcn_input, dcn_input))
            for _ in range(CONFIG['num_cross_layers'])
        ])
        self.cross_biases = nn.ParameterList([
            nn.Parameter(torch.randn(dcn_input))
            for _ in range(CONFIG['num_cross_layers'])
        ])
        
        self.wide_layer = nn.Linear(wide_base, 1)
        
        # Deep
        deep_input = dcn_input
        layers = []
        for out_dim in CONFIG['deep_layers']:
            layers.append(nn.Linear(deep_input, out_dim))
            layers.append(nn.BatchNorm1d(out_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(CONFIG['dropout']))
            deep_input = out_dim
        self.deep_network = nn.Sequential(*layers)
        
        # Final
        final_input = 1 + dcn_input + CONFIG['deep_layers'][-1]
        self.final_bn = nn.BatchNorm1d(final_input)
        self.final_layer = nn.Linear(final_input, 1)
        
        self._init_weights()
    
    def _init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Embedding):
                init.normal_(module.weight, 0, 0.01)
            elif isinstance(module, nn.Linear):
                init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    init.zeros_(module.bias)
        
        for weight in self.cross_weights:
            init.xavier_uniform_(weight)
        for bias in self.cross_biases:
            init.zeros_(bias)
    
    def dcn_forward(self, x):
        x0 = x
        for i in range(CONFIG['num_cross_layers']):
            x = x0 * (x @ self.cross_weights[i]) + self.cross_biases[i] + x
        return x
    
    def forward(self, user_ids, item_ids, source_ids, age_ids, duration_ids, gender_ids,
                item_embeddings, item_stats, user_stats, 
                sequence_item_ids, sequence_mask):
        # Embeddings
        user_emb_w = self.user_emb_wide(user_ids)
        item_emb_w = self.item_emb_wide(item_ids)
        source_emb_w = self.source_emb_wide(source_ids)
        age_emb_w = self.age_emb_wide(age_ids)
        duration_emb_w = self.duration_emb_wide(duration_ids)
        gender_emb_w = self.gender_emb_wide(gender_ids)
        item_meta_emb = self.item_meta_proj(item_embeddings)
        
        user_emb_d = self.user_emb_deep(user_ids)
        item_emb_d = self.item_emb_deep(item_ids)
        source_emb_d = self.source_emb_deep(source_ids)
        age_emb_d = self.age_emb_deep(age_ids)
        duration_emb_d = self.duration_emb_deep(duration_ids)
        gender_emb_d = self.gender_emb_deep(gender_ids)
        
        item_stat_emb = self.item_stat_proj(item_stats)
        user_stat_emb = self.user_stat_proj(user_stats)
        
        # SENet on embeddings
        if CONFIG['use_senet']:
            emb_stack = torch.stack([
                user_emb_d, item_emb_d, source_emb_d, age_emb_d,
                duration_emb_d, gender_emb_d, item_meta_emb
            ], dim=1)  # [batch, 7, emb_size]
            emb_stack = self.senet(emb_stack)
            user_emb_d, item_emb_d, source_emb_d, age_emb_d, duration_emb_d, gender_emb_d, item_meta_emb = \
                emb_stack[:, 0], emb_stack[:, 1], emb_stack[:, 2], emb_stack[:, 3], emb_stack[:, 4], emb_stack[:, 5], emb_stack[:, 6]
        
        # Bilinear interactions
        emb_stack_for_bilinear = torch.stack([
            user_emb_w, item_emb_w, source_emb_w, age_emb_w,
            duration_emb_w, gender_emb_w, item_meta_emb
        ], dim=1)
        bilinear_features = self.bilinear(emb_stack_for_bilinear)
        
        # Multi-head PPM
        sequence_embs = self.item_emb_deep(sequence_item_ids)
        ppm_features = self.ppm(item_emb_d, sequence_embs, sequence_mask)
        
        # Combine
        wide_concat = torch.cat([
            user_emb_w, item_emb_w, source_emb_w, age_emb_w,
            duration_emb_w, gender_emb_w, item_meta_emb,
            item_stat_emb, user_stat_emb, ppm_features,
            bilinear_features
        ], dim=1)
        
        deep_concat = torch.cat([
            user_emb_d, item_emb_d, source_emb_d, age_emb_d,
            duration_emb_d, gender_emb_d, item_meta_emb,
            item_stat_emb, user_stat_emb, ppm_features
        ], dim=1)
        
        wide_out = self.wide_layer(wide_concat)
        cross_out = self.dcn_forward(deep_concat)
        deep_out = self.deep_network(deep_concat)
        
        combined = torch.cat([wide_out, cross_out, deep_out], dim=1)
        combined = self.final_bn(combined)
        output = self.final_layer(combined)
        
        return output

model = UltraEnhancedPPM().to(device)

# Label smoothing loss
class LabelSmoothingBCELoss(nn.Module):
    def __init__(self, smoothing=0.01):
        super().__init__()
        self.smoothing = smoothing
        self.bce = nn.BCEWithLogitsLoss()
    
    def forward(self, pred, target):
        target = target * (1 - self.smoothing) + 0.5 * self.smoothing
        return self.bce(pred, target)

criterion = LabelSmoothingBCELoss(smoothing=CONFIG['label_smoothing'])
optimizer = AdamW(model.parameters(), lr=CONFIG['LR'], weight_decay=CONFIG['weight_decay'])

total_steps = (len(train_data) // CONFIG['BATCH_SIZE'] + 1) * CONFIG['EPOCHS']
scheduler = CosineAnnealingLR(optimizer, T_max=total_steps, eta_min=CONFIG['LR_MIN'])

print(f"  Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"  PPM heads: {CONFIG['ppm_num_heads']}")
print(f"  Sequence length: {CONFIG['sequence_length']}")

# HELPER FUNCTIONS
def get_stats_batch(uids, iids, items_stats, users_stats):
    item_stats_df = items_stats.reindex(iids, fill_value=0.0)
    item_stats_array = item_stats_df.values.astype(np.float32)
    
    user_stats_df = users_stats.reindex(uids, fill_value=0.0)
    user_stats_array = user_stats_df.values.astype(np.float32)
    
    return item_stats_array, user_stats_array

def get_user_sequences_batch(uids, user_sequences, seq_length):
    batch_size = len(uids)
    batch_sequences = np.zeros((batch_size, seq_length), dtype=np.int64)
    batch_masks = np.zeros((batch_size, seq_length), dtype=np.float32)
    
    for idx, uid in enumerate(uids):
        if uid in user_sequences:
            full_seq = user_sequences[uid]['items']
            seq_len = min(len(full_seq), seq_length)
            
            if len(full_seq) < seq_length:
                batch_sequences[idx, -seq_len:] = full_seq
                batch_masks[idx, -seq_len:] = 1
            else:
                batch_sequences[idx] = full_seq[-seq_length:]
                batch_masks[idx] = 1
    
    return batch_sequences, batch_masks

def evaluate_model(model, data, items_stats, users_stats, user_seqs, desc="Validation"):
    model.eval()
    all_preds = []
    all_targets = []
    
    num_batches = (len(data) + CONFIG['BATCH_SIZE'] - 1) // CONFIG['BATCH_SIZE']
    
    with torch.no_grad():
        for batch_idx in tqdm(range(num_batches), desc=desc, leave=False):
            start = batch_idx * CONFIG['BATCH_SIZE']
            end = min(start + CONFIG['BATCH_SIZE'], len(data))
            
            batch = data.iloc[start:end]
            batch_users = batch['uid'].values
            batch_items = batch['iid'].values
            
            batch_users_meta = users_meta.loc[batch_users]
            batch_items_meta = items_meta.loc[batch_items]
            
            targets = batch['target'].values
            
            item_stats_batch, user_stats_batch = get_stats_batch(
                batch_items, batch_users, items_stats, users_stats
            )
            
            seq_items, seq_masks = get_user_sequences_batch(
                batch_users, user_seqs, CONFIG['sequence_length']
            )
            
            user_ids = torch.tensor(batch_users, dtype=torch.long, device=device)
            item_ids = torch.tensor(batch_items, dtype=torch.long, device=device)
            source_ids = torch.tensor(batch_items_meta['sid'].values, dtype=torch.long, device=device)
            age_ids = torch.tensor(batch_users_meta['age'].values, dtype=torch.long, device=device)
            duration_ids = torch.tensor(batch_items_meta['dur'].values, dtype=torch.long, device=device)
            gender_ids = torch.tensor(batch_users_meta['sex'].values, dtype=torch.long, device=device)
            item_embeddings = torch.tensor(np.stack(batch_items_meta['emb'].values), dtype=torch.float32, device=device)
            item_stats_tensor = torch.tensor(item_stats_batch, dtype=torch.float32, device=device)
            user_stats_tensor = torch.tensor(user_stats_batch, dtype=torch.float32, device=device)
            
            sequence_item_ids = torch.tensor(seq_items, dtype=torch.long, device=device)
            sequence_mask = torch.tensor(seq_masks, dtype=torch.float32, device=device)
            
            outputs = model(user_ids, item_ids, source_ids, age_ids, duration_ids, gender_ids,
                          item_embeddings, item_stats_tensor, user_stats_tensor,
                          sequence_item_ids, sequence_mask)
            probs = torch.sigmoid(outputs).squeeze().cpu().numpy()
            
            all_preds.extend(probs)
            all_targets.extend(targets)
    
    auc = roc_auc_score(all_targets, all_preds)
    return auc

# TRAINING LOOP

print("\n[5/9] Training ultra-enhanced model...")
best_val_auc = 0.0
train_num_batches = (len(train_data) + CONFIG['BATCH_SIZE'] - 1) // CONFIG['BATCH_SIZE']

for epoch in range(CONFIG['EPOCHS']):
    print(f"\n{'='*80}\nEPOCH {epoch+1}/{CONFIG['EPOCHS']}\n{'='*80}")
    
    model.train()
    train_loss = 0.0
    
    for batch_idx in tqdm(range(train_num_batches), desc="Training"):
        start = batch_idx * CONFIG['BATCH_SIZE']
        end = min(start + CONFIG['BATCH_SIZE'], len(train_data))
        
        batch = train_data.iloc[start:end]
        batch_users = batch['uid'].values
        batch_items = batch['iid'].values
        
        batch_users_meta = users_meta.loc[batch_users]
        batch_items_meta = items_meta.loc[batch_items]
        
        targets = torch.tensor(batch['target'].values, dtype=torch.float32, device=device).unsqueeze(1)
        
        item_stats_batch, user_stats_batch = get_stats_batch(
            batch_items, batch_users, item_stats, user_stats
        )
        
        seq_items, seq_masks = get_user_sequences_batch(
            batch_users, user_sequences, CONFIG['sequence_length']
        )
        
        user_ids = torch.tensor(batch_users, dtype=torch.long, device=device)
        item_ids = torch.tensor(batch_items, dtype=torch.long, device=device)
        source_ids = torch.tensor(batch_items_meta['sid'].values, dtype=torch.long, device=device)
        age_ids = torch.tensor(batch_users_meta['age'].values, dtype=torch.long, device=device)
        duration_ids = torch.tensor(batch_items_meta['dur'].values, dtype=torch.long, device=device)
        gender_ids = torch.tensor(batch_users_meta['sex'].values, dtype=torch.long, device=device)
        item_embeddings = torch.tensor(np.stack(batch_items_meta['emb'].values), dtype=torch.float32, device=device)
        item_stats_tensor = torch.tensor(item_stats_batch, dtype=torch.float32, device=device)
        user_stats_tensor = torch.tensor(user_stats_batch, dtype=torch.float32, device=device)
        
        sequence_item_ids = torch.tensor(seq_items, dtype=torch.long, device=device)
        sequence_mask = torch.tensor(seq_masks, dtype=torch.float32, device=device)
        
        optimizer.zero_grad()
        outputs = model(user_ids, item_ids, source_ids, age_ids, duration_ids, gender_ids,
                      item_embeddings, item_stats_tensor, user_stats_tensor,
                      sequence_item_ids, sequence_mask)
        loss = criterion(outputs, targets)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), CONFIG['GRAD_CLIP'])
        
        optimizer.step()
        scheduler.step()
        
        train_loss += loss.item()
        
        if batch_idx % 100 == 0:
            torch.cuda.empty_cache()
    
    avg_train_loss = train_loss / train_num_batches
    
    print("\n[6/9] Validating...")
    val_auc = evaluate_model(model, val_data, item_stats, user_stats, user_sequences)
    
    print(f"\n Epoch {epoch+1} Results:")
    print(f"  Train Loss: {avg_train_loss:.6f}")
    print(f"  Val AUC: {val_auc:.6f}")
    
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        torch.save(model.state_dict(), f"{CONFIG['output_folder']}ultra_enhanced_best.pth")
        print(f"  ✓ Best model saved! (AUC: {best_val_auc:.6f})")
    
    torch.cuda.empty_cache()
    gc.collect()

# PREDICTIONS WITH CALIBRATION
print("\n[7/9] Generating predictions with calibration...")
model.load_state_dict(torch.load(f"{CONFIG['output_folder']}ultra_enhanced_best.pth"))
model.eval()

test_num_batches = (len(test) + CONFIG['BATCH_SIZE'] - 1) // CONFIG['BATCH_SIZE']
outputs_list = []

with torch.no_grad():
    for batch_idx in tqdm(range(test_num_batches), desc="Predicting"):
        start = batch_idx * CONFIG['BATCH_SIZE']
        end = min(start + CONFIG['BATCH_SIZE'], len(test))
        
        batch = test.iloc[start:end]
        batch_users = batch['uid'].values
        batch_items = batch['iid'].values
        
        batch_users_meta = users_meta.loc[batch_users]
        batch_items_meta = items_meta.loc[batch_items]
        
        item_stats_batch, user_stats_batch = get_stats_batch(
            batch_items, batch_users, item_stats, user_stats
        )
        
        seq_items, seq_masks = get_user_sequences_batch(
            batch_users, user_sequences, CONFIG['sequence_length']
        )
        
        user_ids = torch.tensor(batch_users, dtype=torch.long, device=device)
        item_ids = torch.tensor(batch_items, dtype=torch.long, device=device)
        source_ids = torch.tensor(batch_items_meta['sid'].values, dtype=torch.long, device=device)
        age_ids = torch.tensor(batch_users_meta['age'].values, dtype=torch.long, device=device)
        duration_ids = torch.tensor(batch_items_meta['dur'].values, dtype=torch.long, device=device)
        gender_ids = torch.tensor(batch_users_meta['sex'].values, dtype=torch.long, device=device)
        item_embeddings = torch.tensor(np.stack(batch_items_meta['emb'].values), dtype=torch.float32, device=device)
        item_stats_tensor = torch.tensor(item_stats_batch, dtype=torch.float32, device=device)
        user_stats_tensor = torch.tensor(user_stats_batch, dtype=torch.float32, device=device)
        
        sequence_item_ids = torch.tensor(seq_items, dtype=torch.long, device=device)
        sequence_mask = torch.tensor(seq_masks, dtype=torch.float32, device=device)
        
        outputs = model(user_ids, item_ids, source_ids, age_ids, duration_ids, gender_ids,
                      item_embeddings, item_stats_tensor, user_stats_tensor,
                      sequence_item_ids, sequence_mask)
        probs = torch.sigmoid(outputs).squeeze().cpu().numpy()
        outputs_list.extend(probs)

# Simple calibration: scale to match training distribution
train_mean = train['target'].mean()
pred_mean = np.mean(outputs_list)
calibration_factor = train_mean / (pred_mean + 1e-8)
outputs_list = np.array(outputs_list) * calibration_factor
outputs_list = np.clip(outputs_list, 0.0, 1.0)

submission = pd.DataFrame({
    'id': test_to_save['uid'].astype(str) + '_' + test_to_save['iid'].astype(str),
    'target': outputs_list
})

submission.to_csv(f"{CONFIG['output_folder']}{CONFIG['test_output_path']}_best.csv", index=False)

print(f"\n{'='*80}\nTRAINING COMPLETE!\n{'='*80}")
print(f"Best Val AUC: {best_val_auc:.6f}")


ULTRA-ENHANCED PPM MODEL
Multi-head PPM + FiBiNet + Advanced Features
Target: 0.650-0.670 AUC

Device: cuda

[1/9] Loading data...
  Train: 122,360,517, Test: 3,572,662

[2/9] Building user sequences...


Building sequences: 100%|██████████| 19/19 [01:10<00:00,  3.71s/it]


  ✓ Built 183,404 sequences in 71.1s

[3/9] Computing enhanced features...
  Item features: 4
  User features: 6
  Train: 116,242,492 | Val: 6,118,025

[4/9] Defining ultra-enhanced model...
  Parameters: 223,352,442
  PPM heads: 4
  Sequence length: 20

[5/9] Training ultra-enhanced model...

EPOCH 1/1


Training: 100%|██████████| 8109/8109 [3:08:49<00:00,  1.40s/it]



[6/9] Validating...



📊 Epoch 1 Results:
  Train Loss: 0.170403
  Val AUC: 0.890975
  ✓ Best model saved! (AUC: 0.890975)

[7/9] Generating predictions with calibration...


Predicting: 100%|██████████| 250/250 [01:47<00:00,  2.33it/s]



TRAINING COMPLETE!
📊 Best Val AUC: 0.890975
🎯 Target: 0.650-0.670 AUC

New Features:
  ✅ Multi-head PPM (4 heads)
  ✅ SENet feature importance
  ✅ FiBiNet bilinear interactions
  ✅ Enhanced statistical features (10 total)
  ✅ Longer sequences (15 items)
  ✅ Label smoothing
  ✅ Prediction calibration
